In [2]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
import requests
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

In [1]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [5]:
# Load .env enviroment variables
load_dotenv("ProjectAPI.env")

True

In [13]:
# The original tickers plus 6 new ones to consider other golbal market, across various sectors compared to the global spy movement  

# Set the tickers
tickers = ["SPY", "IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW", 'XLK','XLV','XLP','ARKK','ICLN','ROBO',]

# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)

In [14]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("Alpaca_API_Key")
alpaca_secret_key = os.getenv("Alpaca_Secret_Key")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [15]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [16]:
# Create current date variable
import datetime
today = datetime.date.today()
historical = today - datetime.timedelta(days=550)

In [17]:
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp(historical, tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

In [19]:
# Set timeframe to 18 months for the Alpaca API
timeframe = "1Day"


In [20]:
#8
# Get 18 months of pricing data for ETF portfolio
df_portfolio = alpaca.get_bars(tickers, timeframe, start=start_date, end=end_date).df

df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-08-19 04:00:00+00:00,44.69,46.410,44.490,151243,46.2400,28454333,45.071206,ARKK
2022-08-22 04:00:00+00:00,43.61,44.790,43.280,137461,43.9600,24679569,43.775314,ARKK
2022-08-23 04:00:00+00:00,43.37,44.600,42.730,110431,43.5001,20800628,43.558235,ARKK
2022-08-24 04:00:00+00:00,44.50,45.150,43.140,88085,43.5800,17435658,44.435014,ARKK
2022-08-25 04:00:00+00:00,45.78,45.780,44.380,77794,45.0400,14731237,45.146808,ARKK
...,...,...,...,...,...,...,...,...
2024-02-13 05:00:00+00:00,142.86,144.470,142.150,96008,143.4700,11170135,142.935492,XLV
2024-02-14 05:00:00+00:00,144.10,144.150,143.075,83201,143.3500,9701053,143.630030,XLV
2024-02-15 05:00:00+00:00,145.15,145.430,143.990,90086,144.2000,9294805,144.959734,XLV
